In [1]:
import torch
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np

In [2]:
from GWO.GWO_optimizer import GWOOptimizer

In [3]:
def sphere(x):
    return torch.sum(x**2).item()

# Bounds per dimension
bounds = [(-5, 5), (0, 10), (-2, 2)]

gwo = GWOOptimizer(sphere, bounds, population_size=10, max_iter=100)
best_pos, losses = gwo.optimize()

print("Best position:", best_pos)
print("Best loss:", losses[-1])

100%|██████████| 100/100 [00:00<00:00, 6906.48it/s]

Best position: tensor([6.8888e-10, 7.0825e-10, 6.3912e-10])
Best loss: 1.384643063156349e-18


In [4]:
data=torch.load("/Users/jaworskj/Grey_wolf_optimizer_torch/sample_1.pth")

In [5]:
data.size()

torch.Size([484, 512, 512])

In [6]:
space_columns=data.flatten(1)
time_columns=space_columns.T

In [7]:
# ------------------------------------------------------------
# INSERT YOUR MATRIX A HERE (torch.Tensor)
# Example placeholder:
# A = torch.randn(200, 200, dtype=torch.float64)
# ------------------------------------------------------------
A = time_columns  # <<< Replace this with your real matrix
assert A is not None, "Please insert your matrix A (torch.Tensor)"

A = A.to(torch.float64)
device = "cuda" if torch.cuda.is_available() else "cpu"
A = A.to(device)

m, n = A.shape

# ------------------------------------------------------------
# OBJECTIVE FUNCTION FOR GWO
# p is a 1D vector to optimize
# ------------------------------------------------------------

def projection_objective(p_flat):
    p = p_flat.to(torch.float64)

    # Normalize projection vector
    p = p / (torch.norm(p) + 1e-12)

    # Compute rank-1 approximation
    y = A @ p        # shape (m,)
    A_hat = y[:,None] @ p[None,:]  # shape (m,n)

    loss = torch.norm(A - A_hat)**2
    return loss.item()


# ------------------------------------------------------------
# SETUP THE OPTIMIZATION BOUNDS
# p has length n (number of columns)
# ------------------------------------------------------------

bounds = [(-1.0, 1.0)] * n  # each parameter ∈ [-1,1]


# ------------------------------------------------------------
# RUN GWO OPTIMIZATION
# ------------------------------------------------------------

gwo = GWOOptimizer(
    objective_function=projection_objective,
    bounds=bounds,
    population_size=12,
    max_iter=300,
    device=device
)

best_p_flat, loss_curve = gwo.optimize()
best_p = best_p_flat.to(torch.float64)
best_p = best_p / (torch.norm(best_p) + 1e-12)


# ------------------------------------------------------------
# RECONSTRUCTION USING OPTIMIZED PROJECTION
# ------------------------------------------------------------

y = A @ best_p
A_hat = y[:,None] @ best_p[None,:]

A_cpu = A.detach().cpu()
A_hat_cpu = A_hat.detach().cpu()

# ------------------------------------------------------------
# LOSS CURVE PLOT
# ------------------------------------------------------------

plt.figure(figsize=(7,5))
plt.plot(loss_curve)
plt.xlabel("Iteration")
plt.ylabel("Reconstruction Loss ||A - A_hat||^2")
plt.title("GWO Optimization Loss Curve")
plt.grid(True)
plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# VISUALIZE ORIGINAL VS RECONSTRUCTION
# (Only if matrix is image-like)
# ------------------------------------------------------------

if m == n and m <= 512:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(A_cpu, cmap="viridis")
    plt.title("Original A")
    plt.colorbar(shrink=0.7)

    plt.subplot(1,2,2)
    plt.imshow(A_hat_cpu, cmap="viridis")
    plt.title("Reconstructed A_hat (Rank-1)")
    plt.colorbar(shrink=0.7)

    plt.tight_layout()
    plt.show()

print("Optimization finished.")
print("Best loss =", loss_curve[-1])


  3%|▎         | 9/300 [00:37<20:23,  4.21s/it]


KeyboardInterrupt: 